``` python
# if you running at azure nootbook to prepare env
#!pip install lxml --ignore-installed lxml
#!pip install tushare==1.2.54 
```

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from greenseer.configuration import do_global_configuration
do_global_configuration("config.properties")

In [3]:
from greenseer.dataset.china_dataset import fetch_default_targets, fetch_train_set, fetch_targets

y = fetch_default_targets()
y.head(5)

,st
code,
605001,0
688233,0
000612,0
300270,0
002369,0


- 600519：茅台

In [4]:
from greenseer.dataset.china_dataset import fetch_multi_report
from logging.config import fileConfig
import numpy as np
fileConfig('logging_config.ini')
reports = fetch_multi_report(stock_ids=np.array(["600519"]))
reports.shape

(57, 242)

In [5]:
subjects = [r'\w*应付\w*|\w*应收\w*|存货|货币资金|\w*生物\w*']

from sklearn.pipeline import Pipeline
from greenseer.preprocessing.transformers import  regular_expression_column_filter, \
    pick_annual_report_china

prepare_pipeline = Pipeline([
        ('pick_annual_report', pick_annual_report_china()),
        ('pick_subject', regular_expression_column_filter(patterns=subjects)),
])

train, test = fetch_train_set()

X_train = prepare_pipeline.fit_transform(train)
X_train.head(5)

专项应付款(万元)  公益性生物资产(万元)  其他应付款(万元)  其他应收款(万元)  内部应付款(万元)  \
603920 2019-12-31        0.0          0.0     4862.0     2776.0        0.0   
       2018-12-31        0.0          0.0     6389.0     1546.0        0.0   
       2017-12-31        0.0          0.0     1318.0     1599.0        0.0   
       2016-12-31        0.0          0.0     1065.0       57.0        0.0   
       2015-12-31        0.0          0.0      762.0       57.0        0.0   

                   内部应收款(万元)   存货(万元)  应付保证金(万元)  应付债券(万元)  应付分保账款(万元)  ...  \
603920 2019-12-31        0.0  23234.0        0.0       0.0         0.0  ...   
       2018-12-31        0.0  21829.0        0.0       0.0         0.0  ...   
       2017-12-31        0.0  20188.0        0.0       0.0         0.0  ...   
       2016-12-31        0.0  13958.0        0.0       0.0         0.0  ...   
       2015-12-31        0.0   9733.0        0.0       0.0         0.0  ...   

                   应收分保账款(万元)  应收利息(万元)  应收票据(万元)  应收股利(万元)  应收补贴款(万元)  \
603920 2019-12-31         0.0       0.0     130.0       0.0        0.0   
       2018-12-31         0.0       0.0       0.0       0.0        0.0   
       2017-12-31         0.0       0.0       0.0       0.0        0.0   
       2016-12-31         0.0       0.0       5.0       0.0        0.0   
       2015-12-31         0.0       0.0       0.0       0.0        0.0   

                   应收账款(万元)  生产性生物资产(万元)  货币资金(万元)  长期应付款(万元)  长期应收款(万元)  
603920 2019-12-31   68684.0          0.0  121725.0        0.0        0.0  
       2018-12-31   56775.0          0.0   84207.0        0.0        0.0  
       2017-12-31   48698.0          0.0   65040.0        0.0        0.0  
       2016-12-31   40542.0          0.0   33601.0        0.0        0.0  
       2015-12-31   30135.0          0.0   27328.0      252.0        0.0  

[5 rows x 31 columns]

In [6]:

target = fetch_targets(index=X_train.index,level=0)